In [1]:
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd


import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_file = pd.read_csv("../TAMU_FINAL_DATASET_2018.csv")

In [3]:
# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

In [5]:
df.shape

(100000, 469)

In [4]:
sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)


df = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)
# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

(2726, 469)
(2726, 469)
(5452, 469)
(4042, 469)
(4042, 468)
(4042,)


In [5]:
# modeling 

x = tf.placeholder(dtype = tf.float32, shape = [None, 468])
y = tf.placeholder(dtype = tf.int32, shape = [None])


# tanh, elu, selu, softplus, and softsign

#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.relu)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.tanh)
#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.selu)


#logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.softplus)
logits = tf.contrib.layers.fully_connected(x, 128, tf.nn.elu)


loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_pred = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("x: ", x)
print("logits: ", logits)
print("loss: ", loss)
print("predicted_labels: ", correct_pred)

x:  Tensor("Placeholder:0", shape=(?, 468), dtype=float32)
logits:  Tensor("fully_connected/Elu:0", shape=(?, 128), dtype=float32)
loss:  Tensor("Mean:0", shape=(), dtype=float32)
predicted_labels:  Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [6]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
        #print('EPOCH', i)
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: X_train, y: y_train})
        if i % 50 == 0:
            print("Loss: ", loss)
        #print('DONE WITH EPOCH')

Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)


In [7]:


predicted = sess.run([correct_pred], feed_dict={x: X_train})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_train, predicted)])

# Calculate the accuracy
accuracy = match_count / len(X_train)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.527


In [8]:


predicted = sess.run([correct_pred], feed_dict={x: X_test})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(y_test, predicted)])

# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.493


In [ ]:
# try some other sklearn model here
from sklearn.svm import SVC

clf = SVC(kernel='rbf', gamma='auto')
clf.fit(X_train, y_train)

print('finish fitting data')


pred_train = clf.predict(X_train)

pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))



In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(n_estimators=100, max_depth=2, criterion='entropy', random_state=0)
clf.fit(X_train, y_train)



print(clf.feature_importances_)


pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

[0.         0.02462455 0.         0.04369402 0.04999585 0.
 0.         0.         0.         0.         0.         0.
 0.00304791 0.01717337 0.         0.         0.         0.
 0.00105303 0.         0.         0.00319535 0.00150197 0.
 0.         0.00312999 0.         0.00083371 0.         0.
 0.00072983 0.         0.         0.00221513 0.         0.00175937
 0.01584784 0.00206072 0.         0.         0.01020055 0.0004557
 0.         0.         0.         0.         0.00040789 0.00207858
 0.         0.         0.         0.01667739 0.         0.
 0.         0.         0.         0.00250473 0.00178289 0.
 0.         0.         0.00087815 0.00414516 0.         0.00643278
 0.         0.00169373 0.00988335 0.00370312 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00090298 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.01234607 0.

In [13]:


from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)



pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))


1.0
0.5937031484257871
